#  Simulation of single compartment Hodgkin-Huxley model of active neurite

Voltage : 
$$V(t+1) = V(t) + \Delta t \cdot (\frac{dV}{dt}) = V(t) + \Delta t \cdot (I_e(t) \cdot - \bar{g}_L (V(t)- E_L) - \bar{g}_{Na} m^3h(V(t)- E_{Na}) - \bar{g}_{K}n^4(V(t)- E_K)) \\
= V(t) + \Delta t \cdot (I_e(t) \cdot - 0.3 (V(t)- 10.6) - 120 m^3h(V(t)- 115) - 36n^4(V(t) + 12))$$ 




Gating Variables: 
$$m(t+1) = m(t) + \Delta t \cdot (\frac{dm}{dt}) = m(t) + \Delta t \cdot (\alpha_m(V(t)) (1-m(t)) - \beta_m(V(t))m(t))$$
$$h(t+1) = h(t) + \Delta t \cdot (\frac{dh}{dt}) = h(t) + \Delta t \cdot (\alpha_h(V(t)) (1-h(t)) - \beta_h(V(t))h(t))$$
$$n(t+1) = n(t) + \Delta t \cdot (\frac{dn}{dt}) = n(t) + \Delta t \cdot (\alpha_n(V(t)) (1-n(t)) - \beta_n(V(t))n(t))$$



In [ ]:
%matplotlib notebook
import math 
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

### Electric Properties and unit test

In [ ]:
delta_t = 0.025 # ms
t_e = 50 # ms
t_s = 300 # ms
I_0 = [0,3,6,8] # muA 


C_m = 1 # muF
E_Na = 115 # mV
E_K = -12 # mV
E_L = 10.6 # mV
E_m = -65 # mV
g_Na = 120 # mS
g_K = 36 # mS
g_L = 0.3 # mS

### Hodgkin-Huxley Formulars

In [ ]:
def delta_V_t(input_function,V,m,h,n,t): 
    return 1/C_m * (input_function(t) - g_L*(V - E_L) - g_Na*m**3*h*(V-E_Na) - g_K*n**4*(V - E_K))

def delta_gate_t(alpha,beta,gate,V): # general equation for all gates
    return alpha(V)*(1-gate) - beta(V)*gate

def alpha_m(V):
    if(V != 25):
        return 0.1 * (V -25)/(1 - math.exp(-(V-25)/10))
    else:
        return 1
    
def beta_m(V):
    return 4*math.exp(-V/18)

def alpha_h(V):
    return 0.07*math.exp(-V/20)

def beta_h(V):
    return 1/(1 + math.exp(-(V-30)/10))

def alpha_n(V):
    if(V!=10):
        return 0.01*((V-10)/(1-math.exp(-(V-10)/10)))
    else:
        return 0.1
    
def beta_n(V):
    return 0.125*math.exp(-V/80)

### Steady State values for gating variables

In [ ]:
voltage = [0]#range(-50,150)
n_stat = np.zeros(len(voltage))
m_stat = np.zeros(len(voltage))
h_stat = np.zeros(len(voltage))
time = np.arange(0,500,0.025)

for i,v in enumerate(voltage):
    n = np.zeros(time.shape[0])
    m = np.zeros(time.shape[0])
    h = np.zeros(time.shape[0])
    
    j = 1
    for t in time[1:]:
        n_old = n[j-1]
        delta_n = delta_gate_t(alpha_n,beta_n,n_old,v)
        n_new = n_old + delta_t * delta_n
        n[j] = n_new 
        
        m_old = m[j-1]
        delta_m = delta_gate_t(alpha_m,beta_m,m_old,v)
        m_new = m_old + delta_t * delta_m
        m[j] = m_new 
        
        
        h_old = h[j-1]
        delta_h = delta_gate_t(alpha_h,beta_h,h_old,v)
        h_new = h_old + delta_t * delta_h
        h[j] = h_new 
        
        j += 1
        
    n_stat[i] = n[len(n)-1] 
    m_stat[i] = m[len(m)-1] 
    h_stat[i] = h[len(h)-1] 
    
    
    
#fig, ax = plt.subplots()
#ax.plot(voltage, n_stat, label = "n") 
#ax.plot(voltage, m_stat, label = "m")
#ax.plot(voltage, h_stat, label = "h")
#ax.legend()
#ax.set(xlabel='Voltage(mV)', ylabel='Magnitude',
#       title='Steady State values for gating variables')

#ax.grid()
#plt.show() 
print(m_stat[0])
print(n_stat[0])
print(h_stat[0])

### Hodgkin-Huxley Approximation

In [ ]:
#------------------------------------------Current Functions-----------------------------------------------------------    
def create_rectangular_current(I):
    def rectangular_current(t):
        if(t< t_e or t_s <= t):
            return 0
        if(t_e <= t and t < t_s):
            return I
    
    return rectangular_current    
    
def approximate_membran_potential(delta_t,t,input_function):
        
        # Initial condition
        time = np.arange(0,t,delta_t)
        steps = time.shape[0]
        
        
        
        V = np.zeros(steps)
        m = np.zeros(steps)
        h = np.zeros(steps)
        n = np.zeros(steps)
        
        m[0] = m_stat[49]
        print(m[0])
        n[0] = n_stat[49]
        h[0] = h_stat[49]
        
        i = 1
        for t in time[1:]:
            
            V_old = V[i-1]
            m_old = m[i-1]
            h_old = h[i-1]
            n_old = n[i-1]
            
            
            delta_m = delta_gate_t(alpha_m,beta_m,m_old,V_old)
            m_new = m_old + delta_t * delta_m
            m[i] = m_new
            
            delta_h = delta_gate_t(alpha_h,beta_h,h_old,V_old)
            h_new = h_old + delta_t * delta_h
            h[i] = h_new 
            
            delta_n = delta_gate_t(alpha_n,beta_n,n_old,V_old)
            n_new = n_old + delta_t * delta_n
            n[i] = n_new
            
            delta_V = delta_V_t(input_function,V_old,m_old,h_old,n_old,t)
            V_new = V_old + delta_t * delta_V
            V[i] = V_new
            
            i += 1
        return V,m,h,n,time
 


for I in I_0:
    V,m,h,n,time = approximate_membran_potential(delta_t,500,create_rectangular_current(I))
    
    fig, ax = plt.subplots()
    ax.plot(time, V)

    ax.set(xlabel='time (ms)', ylabel='voltage (mV)',
       title='Hodgkin-Huxley (HH) model (Input Current: %d muA)' % I)
    ax.grid()
    plt.show()

### Friring Rate Simulation

In [ ]:
t_s = 800
I_0 = np.arange(0,20,0.5)
firing_rates = np.zeros(I_0.shape[0])

def calculate_firing_rate(V):
    rate = 0
    for i in range(1,len(V)-1):
        if(V[i] >= 85 and V[i-1]< 85): # neural threshold = +20mV given a E_m of ca. -65mV --> 85mV threshold
            rate += 1
    
    return rate/(len(V)-(10/0.025)) #

for i,I in enumerate(I_0):
    V,m,h,n,time = approximate_membran_potential(delta_t,1000,create_rectangular_current(I))
    firing_rates[i] = calculate_firing_rate(V)

    
fig, ax = plt.subplots()
ax.plot(I_0, firing_rates)

ax.set(xlabel='Current (muA)', ylabel='Firing Rate',
title='HH - Model Firing Rates (Threshold 85mV)')

ax.grid()
plt.show()
    